# The Price is Right

## Week 8 Order of Play

Day 1: Modal.com and SpecialistAgent  
Day 2: RAG, FrontierAgent, Ensemble Agent  
Day 3: ScannerAgent, MessengerAgent  
Day 4: AutonomousPlannerAgent and DealAgentFramework  
Day 5: The Price Is Right Finale

## RAG (Retrieval Augmented Generation) based on a dataset of 800,000 scraped Amazon products

#### For our 2nd agent, we will be asking OpenAI to estimate the price of one of our deals - and we will give it a hand.

We discovered that LLMs are really good at this, out of the box.

And we discovered that we can beat a frontier LLM by fine-tuning an open-source LLM.

Now we are going to try **inference time** techniques instead of training -- by using RAG!

In [1]:
# imports

import os
import logging
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from litellm import completion
from tqdm.notebook import tqdm
from agents.evaluator import evaluate
from agents.items import Item

In [2]:
# environment

load_dotenv(override=True)
DB = "products_vectorstore"

In [3]:
# Log in to HuggingFace
# If you don't have a HuggingFace account, you can set one up for free at www.huggingface.co
# And then add the HF_TOKEN to your .env file as explained in the project README

hf_token = os.environ['HF_TOKEN']
login(token=hf_token, add_to_git_credential=False)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
LITE_MODE = True

In [15]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


# Now create a Chroma Datastore

Now we will use the free, open-source Vector database Chroma.  
We will create a Chroma datastore with 400,000 products from our training dataset.

In [5]:
client = chromadb.PersistentClient(path=DB)

# Introducing the SentenceTransformer Encoding LLM

The all-MiniLM is a very useful model from HuggingFace that maps sentences & paragraphs to 384 dimensional vectors and is ideal for tasks like semantic search.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

It can run pretty quickly locally.

As an alternative, OpenAI provides a closed-source Embeddings model. Benefits compared to OpenAI embeddings:
1. It's free and fast!
3. We can run it locally, so the data never leaves our box - might be useful if you're building a personal RAG

In [6]:
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [8]:
# Pass in a list of texts, get back a numpy array of vectors

vector = encoder.encode(["A proficient AI engineer who has almost reached the finale of AI Engineering Core Track!"])[0]
print(vector.shape)
vector

(384,)


array([-5.68233915e-02, -6.70465827e-02,  4.41129506e-02,  5.98607305e-03,
       -2.28948668e-02, -2.95300353e-02,  5.56368791e-02,  3.42665240e-02,
       -1.08529389e-01, -3.81690413e-02, -7.43872598e-02, -1.03664272e-01,
        1.69147812e-02,  1.33929204e-03, -6.86191246e-02,  8.99353623e-02,
       -1.45186214e-02, -2.43885256e-02,  4.21854435e-03, -9.62992907e-02,
       -2.51799170e-02,  4.60675657e-02,  4.95350361e-03, -3.88680100e-02,
        1.07717060e-03,  6.82337359e-02, -1.13860117e-02, -5.83416522e-02,
       -1.03801200e-02, -1.74953435e-02, -1.86478421e-02,  4.07058327e-03,
        1.59438271e-02,  6.49722666e-02,  3.71175855e-02,  2.78224908e-02,
       -4.41945344e-02, -2.34372523e-02,  9.71036330e-02, -5.06139211e-02,
       -1.93864480e-02, -3.83471511e-02,  4.76066805e-02, -3.36107500e-02,
        5.08287288e-02,  3.57934907e-02,  2.91815470e-03, -1.06529139e-01,
        4.07211930e-02, -5.85441478e-04, -1.05607428e-01, -1.03584342e-01,
        3.71124037e-02, -

## With that background, let's populate our Chroma database

### By calculating vectors for 800,000 scraped products

This takes 30 minutes on my machine on my GPU - it might take longer for you - feel free to use the Lite dataset!

In [7]:
# Check if the collection exists; if not, create it

collection_name = "products"
existing_collection_names = [collection.name for collection in client.list_collections()]

if collection_name not in existing_collection_names:
    collection = client.create_collection(collection_name)
    for i in tqdm(range(0, len(train), 1000)):
        documents = [item.summary for item in train[i: i+1000]]
        vectors = encoder.encode(documents).astype(float).tolist()
        metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
        ids = [f"doc_{j}" for j in range(i, i+1000)]
        ids = ids[:len(documents)]
        collection.add(ids=ids, documents=documents, embeddings=vectors, metadatas=metadatas)

collection = client.get_or_create_collection(collection_name)

In [8]:
print(collection.count())

20000


# Let's visualize the vectorized data

In [10]:
# It is very fun turning this up to 800_000 and seeing the full dataset visualized,
# but it almost crashes my box every time so do that at your own risk!! 10_000 is safe!

MAXIMUM_DATAPOINTS = 10_000

In [11]:
CATEGORIES = ['Appliances', 'Automotive', 'Cell_Phones_and_Accessories', 'Electronics','Musical_Instruments', 'Office_Products', 'Tools_and_Home_Improvement', 'Toys_and_Games']
COLORS = ['cyan', 'blue', 'brown', 'orange', 'yellow', 'green' , 'purple', 'red']

In [15]:
# Prework
result = collection.get(include=['embeddings', 'documents', 'metadatas'], limit=MAXIMUM_DATAPOINTS)
vectors = np.array(result['embeddings'])
documents = result['documents']
categories = [metadata['category'] for metadata in result['metadatas']]
colors = [COLORS[CATEGORIES.index(c)] for c in categories]

In [16]:
# Let's try a 2D chart
# TSNE stands for t-distributed Stochastic Neighbor Embedding - it's a common technique for reducing dimensionality of data

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [17]:
# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=4, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vectorstore Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [18]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [19]:
# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=2, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [20]:
test[0]

<Old Blood Noise Excess V2 Distortion Chorus/Delay Pedal = $219.0>

In [9]:
def vector(item):
    return encoder.encode(item.summary)

In [10]:
def find_similars(item):
    vec = vector(item)
    results = collection.query(query_embeddings=vec.astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [23]:
find_similars(test[0])

(['Title: Digitech DOD Carcosa Fuzz Distortion Pedal  \nCategory: Electronics  \nBrand: DigiTech  \nDescription: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  \nDetails: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑mid boost, and a 9‑V power supply.',
  'Title: EarthQuaker Devices Plumes Small Signal Shredder – Purple Sparkle  \nCategory: Guitar Pedals  \nBrand: EarthQuaker Devices  \nDescription: An all‑analog overdrive pedal that delivers three distinct clipping voices and expansive headroom for crystal‑clear, three‑dimensional tone.  \nDetails: Features Flexi‑Switch relay‑based bypass, 9\u202fV power input, and a finely tuned tone control for sculpting low, mid, and high frequencies.',
  'Title: Dunlop Rotovibe Pedal  \nCategory: Audio Effects  \nBrand: Dunlop  \nDescription: A versatile chorus/vibrato pedal that simulates rotating speaker sounds w

In [11]:
# We need to give some context to GPT-5.1 by selecting 5 products with similar descriptions

def make_context(similars, prices):
    message = "For context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [25]:
documents, prices = find_similars(test[0])
print(make_context(documents, prices))

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Digitech DOD Carcosa Fuzz Distortion Pedal  
Category: Electronics  
Brand: DigiTech  
Description: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  
Details: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑mid boost, and a 9‑V power supply.
Price is $159.99

Potentially related product:
Title: EarthQuaker Devices Plumes Small Signal Shredder – Purple Sparkle  
Category: Guitar Pedals  
Brand: EarthQuaker Devices  
Description: An all‑analog overdrive pedal that delivers three distinct clipping voices and expansive headroom for crystal‑clear, three‑dimensional tone.  
Details: Features Flexi‑Switch relay‑based bypass, 9 V power input, and a finely tuned tone control for sculpting low, mid, and high frequencies.
Price is $99.00


In [12]:
def messages_for(item, similars, prices):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}\n\n"
    message += make_context(similars, prices)
    return [{"role": "user", "content": message}]

In [27]:
documents, prices = find_similars(test[0])
print(messages_for(test[0], documents, prices)[0]['content'])

Estimate the price of this product. Respond with the price, no explanation

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Digitech DOD Carcosa Fuzz Distortion Pedal  
Category: Electronics  
Brand: DigiTech  
Description: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  
Details: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑

In [13]:
# The function for gpt-5-mini

def gpt_5__1_rag(item):
    documents, prices = find_similars(item)
    response = completion(model="gpt-5.1", messages=messages_for(item, documents, prices), reasoning_effort="none", seed=42)
    return response.choices[0].message.content

In [16]:
# How much does our favorite distortion pedal cost?

test[0].price

219.0

In [30]:
# Let's do this!!

gpt_5__1_rag(test[0])

'$229.99'

In [ ]:
evaluate(gpt_5__1_rag, test)

In [19]:
import modal
Pricer = modal.Cls.from_name("pricer-service", "Pricer")
pricer = Pricer()

In [17]:
def specialist(item):
    return pricer.price.remote(item.summary)


In [18]:
def get_price(reply):
    reply = reply.replace("$", "").replace(",", "")
    match = re.search(r"[-+]?\d*\.\d+|\d+", reply)
    return float(match.group()) if match else 0

## Download the Neural Network weights from Week 6 into this directory

The file `deep_neural_network.pth` here:

https://drive.google.com/drive/folders/1uq5C9edPIZ1973dArZiEO-VE13F7m8MK?usp=drive_link

In [ ]:

from agents.deep_neural_network import DeepNeuralNetworkInference

runner = DeepNeuralNetworkInference()
runner.setup()
runner.load("deep_neural_network.pth")

def deep_neural_network(item):
    return runner.inference(item.summary)

In [ ]:
def ensemble(item):
    price1 = get_price(gpt_5__1_rag(item))
    price2 = specialist(item)
    price3 = deep_neural_network(item)
    return price1 * 0.8 + price2 * 0.1 + price3 * 0.1


In [ ]:
evaluate(ensemble, test)

In [ ]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [20]:
from agents.frontier_agent import FrontierAgent

agent = FrontierAgent(collection)
agent.price("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")

119.99

In [ ]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

In [ ]:
from agents.neural_network_agent import NeuralNetworkAgent
agent = NeuralNetworkAgent()


In [ ]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

In [ ]:
from agents.ensemble_agent import EnsembleAgent
agent = EnsembleAgent(collection)

In [ ]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")